In [1]:
from selenium import webdriver
import pandas as pd
import openpyxl as op
import time
import re
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np
get_number = re.compile('.*(?=\n)')
get_percent = re.compile('.*(?=%)')
get_url_without_slash = re.compile(".*[0-9a-z]")
password = !echo %password%

In [2]:
def reset():
    driver.get("https://analytics.google.com/analytics/web/?authuser=0#management/Settings/a10587421w80067081p84826225/")

def login():
    driver.get("https://analytics.google.com/analytics/web/?authuser=0#management/Settings/a10587421w80067081p84826225/")
    #아이디입력
    driver.find_element_by_css_selector('#Email').send_keys('taehun9312@gmail.com')
    driver.find_element_by_css_selector('#next').click()
    time.sleep(1)
    #비밀번호입력
    driver.find_element_by_css_selector('#Passwd').send_keys(password[0])
    driver.find_element_by_css_selector('#signIn').click()
    
def login2():
    driver.get("https://analytics.google.com/analytics/web/?authuser=0#management/Settings/a10587421w80067081p84826225/")
    #아이디입력
    driver.find_element_by_css_selector('#identifierId').send_keys('taehun9312@gmail.com')
    driver.find_element_by_css_selector('.RveJvd.snByac').click()
    time.sleep(1)
    #비밀번호입력
    driver.find_element_by_css_selector('.whsOnd.zHQkBf').send_keys(password[0])
    driver.find_element_by_css_selector('.RveJvd.snByac').click()
    
def s_to_i(value):
    value = value.replace(",","")
    value = int(value)
    return value

def check(compared_value):
    if type(compared_value) == list:
        for value in compared_value:
            if value in global_value: raise ValueError
    elif compared_value in global_value: raise ValueError
    else: print("No_duplication")
        
def get_pure_url(url):
    splited_url = url.split("/%")
    url = splited_url[0]
    url = get_url_without_slash.findall(url)[0]
    return url

def get_number_item(unprocessed_number):
    processed_number = unprocessed_number.split("(")[0]
    return s_to_i(processed_number)

        
def calculate_date(origin_date, interval, multiple_number):
    date_set = _get_date(origin_date)
    d1 = date(date_set["year"], date_set["month"], date_set["day"])
    if interval == "month":
        d2 = d1 + relativedelta(months=+1*multiple_number)
    else:
        d2 = d1 + timedelta(days = interval*multiple_number)
    next_date = _to_date(d2)
    return next_date

def _get_date(date):
    date = str(date)
    year = "20" + date[0:2]
    month = date[2:4]
    day = date[4:6]
    if month[0] == "0":
        month = month.replace("0","")
    return {"year": int(year), "month": int(month), "day": int(day)}

def _to_date(date):
    year = str(date.year)[2:4]
    if date.month < 10:
        month = "0" + str(date.month)
    else:
        month = str(date.month)
    if date.day < 10:
        day = "0" + str(date.day)
    else:    
        day = str(date.day)
    return year+month+day

In [3]:
class GA:
    
    def __init__(self, url):
        global start
        global end 
        self.df = pd.DataFrame()
        self.date = "/%3F_u.date00%3D20{start}%26_u.date01%3D20{end}".format(start= start, end=end)
        self.original_url = url
        self.url = get_pure_url(url) + self.date
        self.start = start
        self.end = end
        self.retry = 40
        self.number = 1
        self.df_start = str(start)
        self.df_end = str(end)
        self.dic_values = {}
        self.seg = "0"
        driver.get(self.url)
        
    def set_date(self, interval, number):
        self.interval = interval
        self.number = number

        
    def set_segment_url(self, seg, keep):
        if keep == "on": self.seg = seg
        self.url = get_url_without_slash.findall(self.url)[0] + seg
        self.df = self.df.transpose()
        #"%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ"
        
    def set_tab(self, n_tab):
        self.tab = n_tab
        
    def set_value(self, n_value):
        if type(n_value) != list:
            self.value = [n_value]
        else:
            self.value = n_value
        
    def set_retry(self, n):
        self.retry = n
    
    def go_changed_url(self, n_try):
        start = calculate_date(self.start, self.interval, n_try)
        end = calculate_date(self.end, self.interval, n_try)
        self.df_start = str(start)
        self.df_end = str(end)
        self.date =  "/%3F_u.date00%3D20{start}%26_u.date01%3D20{end}".format(start= start, end=end)
        if len(self.seg) > 1:
            self.url = get_pure_url(self.original_url) + self.date + self.seg
        else:
            self.url = get_pure_url(self.original_url) + self.date
        driver.get(self.url)
        
    def get(self, numbers):
        for n_try in range(1, self.number+1):
            if type(numbers) == int:
                value = self._come_to_me(numbers)
                return value
            else:
                for n in numbers:
                    self.dic_values = self._fill_dic_values(n, self.dic_values)
            for title, value in self.dic_values.items():
                    self.df.loc[self.df_start+" - "+self.df_end, title] = value
            if self.number > 1 and n_try < self.number: 
                    self.dic_values = {}
                    self.go_changed_url(n_try)  
        self.df = self.df.transpose()
 

    def _fill_dic_values(self, numbers, dic_values):
            n = numbers
            value = self._come_to_me(n)
            titles = self._get_title(self.value[n-1])
            
            if type(value) != list:
                if type(self.value[n-1]) != list:
                    #title 1, value 1
                    title = titles[0]
                    if title in dic_values:
                        title = title + str(n)
                    dic_values[title] = value
                else:
                    #titile more, value 1
                    nth = self.value[n-1][1]
                    title = titles[nth-1]
                    if title in dic_values:
                        title = title + str(n)
                    dic_values[title] = value
            else:
                #title more, value more
                for i, v in enumerate(value):
                    title = titles[i]
                    if title in dic_values:
                        title = title + str(n)
                    dic_values[title] = v 
            return dic_values
    
    def _come_to_me(self, n):
        global global_value
        error_check = "on"
        self._click_tab(n)
        value_spec = self.value[n-1]

        if type(value_spec) == list: 
            number = value_spec[0] 
            nth = value_spec[1]
        else: 
            number = value_spec
            nth = value_spec

        if type(value_spec) == list or value_spec == 1:
            value = self._get_value(nth, error_check)
            error_check = "off"
            value_for_compare = self._get_values(number, error_check)
        else:
            value_for_compare = self._get_values(number, error_check)
            value = value_for_compare
        global_value = value_for_compare
        return value

    def _get_title(self, n):
        arr_title = []
        unprocessed_titles = driver.find_elements_by_css_selector("._GArO")
        for title in unprocessed_titles:
            arr_title.append(title.text)
        return arr_title
    
    def get_percent(self, n):
        arr_value = driver.find_elements_by_css_selector("._GAj8")
        percent = arr_value[n-1].text
        processed_percent = float(get_percent.findall(percent)[0])
        print(processed_percent)
        return processed_percent
        
            
#     def get_item(self):
#         for i in range(0, self.retry):
#             try:
#                 #Screen name, Event lbael.. etc
#                 target_name = driver.find_elements_by_css_selector("._GAkub")[0].text
#                 #Unique screen view, screen view.. etc
#                 table_heads = driver.find_elements_by_css_selector("._GAYh.ACTION-sort")
#                 arr_table_heads = []
#                 for table_head in table_heads:
#                     arr_table_heads.append(table_head.text)
#                 num_cols = len(arr_table_heads)
#                 arr_table_heads.insert(0,target_name)
#                 #home_banner_1, home_banner_2.. etc
#                 target_list = driver.find_elements_by_css_selector("._GApu")
#                 #700, 600.. etc
#                 dataset = driver.find_elements_by_css_selector("._GAtjb")
#                 self.df = pd.DataFrame(columns = arr_table_heads)
#                 for row_num, target in enumerate(target_list):
#                     self.df[target_name, target] =
#                         for i in range(0, num_cols+1):
#                             df.loc[item,dataset[row_num*num_cols+i]
                        
#                         value = get_number_item(dataset[i*2+nth-1].text)
#                         check(value)
#                         print("It's ok")
#                         arr_data = []
#                         for data in dataset:
#                             arr_data.append(data.text)
#                         global_value = arr_data
#                         print(value)
#                         return value
#                 time.sleep(1)
#             except:
#                 time.sleep(4)
#                 print("It's not ok")
#                 continue
                        

    #n번째 값 가져오기
    def _get_value(self, n, error_check):
        for i in range(0, self.retry):
            try:
                print(i)
                value = driver.find_element_by_css_selector(".ID-stage{n}-top._GAhFb > ._GAXm > ._GAoCb".format(n = n)).text
                if n != 1: value = get_number.findall(value)[0]
                value = s_to_i(value)
                print("value is", value)
                if error_check == "on":
                    check(value)
                print("It's ok")
                break
            except:
                time.sleep(4)
                print("It's not ok")
                continue
        print(value)
        return value
    
    #n개의 값 가져오기
    def _get_values(self, range_n, error_check):
        arr_value = []
        for n in range(1, range_n+1):
            value = self._get_value(n, error_check)
            arr_value.append(value)
        print(arr_value)
        return arr_value

    #커스텀 리포트 내 탭
    def _click_tab(self, n):
        if n == 1:
            driver.get(self.url)
        else:
            for i in range(0, self.retry):
                try:
                    driver.find_elements_by_css_selector("._GAit._GAl")[n-1].click()
                    break
                except:

                    time.sleep(3)
                    print("It retry to click")
                    continue

In [4]:
# tab_category = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/C02IOMc8QBmCo6k8wf6aGQ/a10587421w80067081p102782120/%3F_u.date00%3D20170424%26_u.date01%3D20170425%26_.useg%3Dbuiltin13%26_r.tabId%3D3915/")
# tab_category.set_tab(5)
# tab_category.set_value([3,3,4,4])
# tab_category.set_date(1,30)
# tab_category.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "on")
# tab_category.get([1,2,3,4])

In [5]:
# excel = op.load_workbook(r"C:\Users\joseph_th\Desktop\Dashboard\Dashboard_for_GA.xlsx")
# s1 = excel["여행 Traffic overview"]["AC"]
# s2 = excel["여행 딜"]["W"]
# s3 = excel["실시간항공"]["Z"]
# s4 = excel["실시간호텔"]["U"]
# s5 = excel["APP 홈 기여"]["S"]
# excel.save(r"C:\Users\joseph_th\Desktop\Dashboard\Dashboard_for_GA_result.xlsx")

In [6]:
driver = webdriver.Chrome(executable_path=r"C:\Users\joseph_th\Downloads\chromedriver_win32\chromedriver")
login2()

In [12]:
global_value = []
start = 170508
end = 170514
reset()

In [11]:
#여행홈 이용률_And 
Tour_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/BNAhfOhCRYGSL64VQWI9pQ/a10587421w80067081p102782120/%3F_u.date00%3D20170410%26_u.date01%3D20170410%26_.useg%3Duser3pWJ7sIwS-mDV3x54DAbeg/")
Tour_1.set_tab(5)
Tour_1.set_value([2,[2,2],[2,2],[2,2],[2,2]])
Tour_1.set_segment_url("%26_.useg%3Duser3pWJ7sIwS-mDV3x54DAbeg", "on") #4.1.9v ↑
Tour_1.set_date(1,3)
Tour_1.get([1,2,4,5])

Tour_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/hT5jydKhQni5if6X7sW0rQ/a10587421w80067081p102782120/%3F_u.date00%3D20170410%26_u.date01%3D20170410%26_.useg%3Duser3pWJ7sIwS-mDV3x54DAbeg/")
Tour_2.set_tab(4)
Tour_2.set_value([[2,2],[2,2],[2,2],[2,2]])
Tour_2.set_segment_url("%26_.useg%3Duser3pWJ7sIwS-mDV3x54DAbeg", "on")
Tour_2.set_date(1,3)
Tour_2.get([1,2,3,4])

0
It's not ok
1
value is 22686
No_duplication
It's ok
22686
0
value is 9900
No_duplication
It's ok
9900
[22686, 9900]
0
value is 9900
It's not ok
1
value is 5344
No_duplication
It's ok
5344
0
value is 22686
It's ok
22686
0
value is 5344
It's ok
5344
[22686, 5344]
0
value is 5344
It's not ok
1
value is 2053
No_duplication
It's ok
2053
0
value is 22686
It's ok
22686
0
value is 2053
It's ok
2053
[22686, 2053]
0
value is 2053
It's not ok
1
value is 938
No_duplication
It's ok
938
0
value is 22686
It's ok
22686
0
value is 938
It's ok
938
[22686, 938]
0
value is 22686
It's not ok
1
value is 22769
No_duplication
It's ok
22769
0
value is 9909
No_duplication
It's ok
9909
[22769, 9909]
0
value is 9909
It's not ok
1
value is 5457
No_duplication
It's ok
5457
0
value is 22769
It's ok
22769
0
value is 5457
It's ok
5457
[22769, 5457]
0
value is 5457
It's not ok
1
value is 2078
No_duplication
It's ok
2078
0
value is 22769
It's ok
22769
0
value is 2078
It's ok
2078
[22769, 2078]
0
value is 2078
It's not

In [15]:
pd.concat([Tour_1.df, Tour_2.df])

,170512 - 170512,170513 - 170513,170514 - 170514
여행 홈,22686.0,22769.0,24076.0
1. 플릭,9900.0,9909.0,10402.0
2. 카테고리 클릭,5344.0,5457.0,6028.0
"4. 실시간 항공, 호텔 검색클릭",2053.0,2078.0,2206.0
5. 기획전 배너 클릭,938.0,990.0,1089.0
테마 컨텐츠,421.0,518.0,518.0
운영 컨텐츠,1016.0,1074.0,1172.0
할인 호텔,142.0,139.0,153.0
지역/나라딜,1111.0,1235.0,1299.0


In [13]:
#국내, 해외, 항공호텔, 전체카테고리
And_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/XAzFnmCtTl6SM1Cmc9OVNw/a10587421w80067081p102782120/")
And_1.set_tab(5)
And_1.set_value([1,1,1,1])
And_1.get([1,2,3,4])

#티몬 전체 카테고리
And_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/gmdw7utOS6OyMdXRs_jizg/a10587421w80067081p102782120/")
And_2.set_tab(1)
And_2.set_value(1)
And_2.get([1])

# #국내, 해외, 여행전체 딜
# And_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/c9agY1srRx2PaD8JAkK0yw/a10587421w80067081p102782120/")
# And_3.set_tab(3)
# And_3.set_value([3,3,3])
# And_3.get([1,2,3])


# And_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "nope")
# And_3.set_value([[3,3],[3,3],[3,3]])
# And_3.get([1,2,3])


#항공검색 해외, 국내, 해외->호텔메인, 해외->국내항공메인
And_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/5fyYOU-QTN6tQSwRlObkUw/a10587421w80067081p102782120/%3F_u.date00%3D20170101%26_u.date01%3D20170301%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ/")
And_4.set_tab(4)
And_4.set_value([2,2,[2,2],[2,2]])
And_4.get([1,2,3,4])

#항공예약 클릭 국내, 해외 
And_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/_yDrAZzXTaeMOGlYkwqqbw/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D%26_r.tabId%3D6028/")
And_5.set_tab(2)
And_5.set_value([1,1])
And_5.get([1,2])

#B.com 카테고리/시스템 큐레이션
And_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/urNzazUpRDu9q-17-qtmtw/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D/")
And_6.set_tab(5)
And_6.set_value([2,1,1,2,1])
And_6.get([1,2,3,4,5])

#B.com 매뉴얼 큐레이션/이벤트
And_7 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/qWyiPek-Tsqgkok-rZsIjA/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D/")
And_7.set_tab(2)
And_7.set_value([2,2])
And_7.get([1,2])

#홈 -> 딜상세 국내, 해외, 카테고리숏컷
And_8 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/4rkbjB7zTGmgcRAlJaVzrg/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D/")
And_8.set_tab(3)
And_8.set_value([2,[2,2],1])
And_8.get([1,2,3])

#홈 -> 콜렉션(국내), 해외
And_9 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/lUOk-ms-R966phrt04_yfA/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D/")
And_9.set_tab(2)
And_9.set_value([2,[2,2]])
And_9.get([1,2])


#여행탭 -> 카테고리, 항공검색결과
And_10 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/rUsbr2MuTou-MkNzI27mfw/a10587421w80067081p102782120/")
And_10.set_tab(3)
And_10.set_value([2,[2,2]])
And_10.get([1,2])

0
It's not ok
1
value is 86310
No_duplication
It's ok
86310
0
value is 86310
It's ok
86310
[86310]
0
value is 86310
It's not ok
1
value is 50436
No_duplication
It's ok
50436
0
value is 50436
It's ok
50436
[50436]
0
value is 50436
It's not ok
1
value is 32830
No_duplication
It's ok
32830
0
value is 32830
It's ok
32830
[32830]
0
value is 32830
It's not ok
1
value is 154800
No_duplication
It's ok
154800
0
value is 154800
It's ok
154800
[154800]
0
value is 154800
It's not ok
1
value is 512484
No_duplication
It's ok
512484
0
value is 512484
It's ok
512484
[512484]
0
value is 512484
It's not ok
1
value is 370803
No_duplication
It's ok
370803
0
value is 44509
No_duplication
It's ok
44509
0
value is 12910
No_duplication
It's ok
12910
[370803, 44509, 12910]
0
value is 370803
It's not ok
1
value is 118287
No_duplication
It's ok
118287
0
value is 12481
No_duplication
It's ok
12481
0
value is 2402
No_duplication
It's ok
2402
[118287, 12481, 2402]
0
value is 118287
It's not ok
1
value is 471822
No_

In [19]:
#국내, 해외, 항공호텔, 전체카테고리
iOS_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/f6fCST7zQwalCMN1dpSuPw/a10587421w80067081p88823264/")
iOS_1.set_tab(5)
iOS_1.set_value([1,1,1,1])
iOS_1.get([1,2,3,4])


#티몬 전체 카테고리
iOS_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/p29BV_8iRc2OovJCvhcg_Q/a10587421w80067081p88823264/%3F_u.date00%3D20170327%26_u.date01%3D20170402/")
iOS_2.set_tab(1)
iOS_2.set_value(1)
iOS_2.get([1])

# #국내딜, 해외딜, 전체딜
# iOS_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/A9m3wyZgQTmxJrO3lQ6UMw/a10587421w80067081p88823264/%3F_u.date00%3D20170327%26_u.date01%3D20170402%26_r.tabId%3D4646%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ/")
# iOS_3.set_tab(3)
# iOS_3.set_value([3,3,3])
# iOS_3.get([1,2,3])

# iOS_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "nope")
# iOS_3.set_value([[3,3],[3,3],[3,3]])
# iOS_3.get([1,2,3])

#항공검색 해외, 국내, 해외 -> 호텔메인, 해외 -> 국내메인
iOS_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/2K60xdJeT2isKsPe4tgpEQ/a10587421w80067081p88823264/%3F_u.date00%3D20170301%26_u.date01%3D20170320/")
iOS_4.set_tab(4)
iOS_4.set_value([2,2,[2,2],[2,2]])
iOS_4.get([1,2,3,4])

#항공예약클릭 국내, 해외
iOS_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/btj6gUJmQ2-NZsSB1vB3tw/a10587421w80067081p88823264/%3F_u.date00%3D20170301%26_u.date01%3D20170331%26_.useg%3D/")
iOS_5.set_tab(2)
iOS_5.set_value([1,1])
iOS_5.get([1,2])

#B.com 카테고리/시스템 큐레이션
iOS_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/mzJgxll5SXqATSN_oHrUiQ/a10587421w80067081p88823264/")
iOS_6.set_tab(5)
iOS_6.set_value([2,1,1,2,1])
iOS_6.get([1,2,3,4,5])

#B.com 매뉴얼 큐레이션/이벤트
iOS_7 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/tZRMJC4JRhm_X9Ql6C4KOg/a10587421w80067081p88823264/")
iOS_7.set_tab(2)
iOS_7.set_value([2,2])
iOS_7.get([1,2])

#홈 -> 딜상세(국내), 해외, 여행카테고리숏컷
iOS_8 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/GDfZCJv9RSC7dYyccZZ1lA/a10587421w80067081p88823264/")
iOS_8.set_tab(3)
iOS_8.set_value([2,[2,2],1])
iOS_8.get([1,2,3])

#홈 -> 콜렉션(국내), 해외
iOS_9 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/9rdCrGeYTaCIuaCGnfLoRQ/a10587421w80067081p88823264/")
iOS_9.set_tab(2)
iOS_9.set_value([2,[2,2]])
iOS_9.get([1,2])

#여행홈 -> 
iOS_10 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/f30PcWxBRsWzVzv0emSDdw/a10587421w80067081p88823264/")
iOS_10.set_tab(3)
iOS_10.set_value([2,[2,2]])
iOS_10.get([1,2])

0
It's not ok
1
value is 3018
No_duplication
It's ok
3018
0
value is 233
No_duplication
It's ok
233
[3018, 233]
0
value is 3018
It's not ok
1
value is 0
No_duplication
It's ok
0
0
value is 0
It's ok
0
[0]
0
value is 0
It's not ok
1
value is 860
No_duplication
It's ok
860
0
value is 860
It's ok
860
[860]
0
value is 860
It's not ok
1
value is 2288
No_duplication
It's ok
2288
0
value is 1517
No_duplication
It's ok
1517
[2288, 1517]
0
value is 2288
It's not ok
1
value is 0
No_duplication
It's ok
0
0
value is 0
It's ok
0
[0]
0
value is 0
It's not ok
1
value is 1639
No_duplication
It's ok
1639
0
value is 454
No_duplication
It's ok
454
[1639, 454]
0
value is 1639
It's not ok
1
value is 709
No_duplication
It's ok
709
0
value is 153
No_duplication
It's ok
153
[709, 153]
0
value is 709
It's not ok
1
value is 2312903
No_duplication
It's ok
2312903
0
value is 2852
No_duplication
It's ok
2852
[2312903, 2852]
0
value is 2852
It's not ok
1
value is 779
No_duplication
It's ok
779
0
value is 2312903
It

In [20]:
#국내카테고리, 해외, 항공호텔, 전체
Mo_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/V-T6iNbTQWi5USC2dSi5tA/a10587421w80067081p89357101/")
Mo_1.set_tab(5)
Mo_1.set_value([1,1,1,2])
Mo_1.get([1,2,3,4])

#티몬 홈 전체
Mo_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/zoovm6xETN-Cd6JF9s2_hQ/a10587421w80067081p89357101/")
Mo_2.set_tab(1)
Mo_2.set_value(1)
Mo_2.get([1])

# #여행딜 전체, 국내, 해외
# Mo_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/upJUz96ERVGUIE2WWd8Tgg/a10587421w80067081p89357101/")
# Mo_3.set_tab(3)
# Mo_3.set_value([3,3,3])
# Mo_3.get([1,2,3])
# Mo_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "nope")
# Mo_3.set_value([[3,3],[3,3],[3,3]])
# Mo_3.get([1,2,3])

#항공검색 해외, 국내, 해외->호텔, 해외->국내
Mo_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/7aMTsZECQu2GVnxwp3IXRg/a10587421w80067081p89357101/")
Mo_4.set_tab(4)
Mo_4.set_value([2,2,[2,2],[2,2]])
Mo_4.get([1,2,3,4])

#항공예약클릭 국내, 해외
Mo_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/Shs74NpmR1Wcy5isHt_ZGA/a10587421w80067081p89357101/")
Mo_5.set_tab(2)
Mo_5.set_value([1,1])
Mo_5.get([1,2])

#부킹닷컴 큐레이션, 호텔메인, 이벤트
Mo_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/W52iAReKSeydMYwmwkpBbA/a10587421w80067081p89357101/")
Mo_6.set_tab(3)
Mo_6.set_value([2,2,2])
Mo_6.get([1,2,3])

#  홈화면 -> 딜상세
Mo_7 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/tbyK5vXiTnuyfCcVRNtBJQ/a10587421w80067081p89357101/")
Mo_7.set_tab(2)
Mo_7.set_value([2,[2,2]])
Mo_7.get([1,2])

0
value is 61405
It's not ok
1
value is 19582
No_duplication
It's ok
19582
0
value is 19582
It's ok
19582
[19582]
0
value is 19582
It's not ok
1
value is 8252
No_duplication
It's ok
8252
0
value is 8252
It's ok
8252
[8252]
0
value is 8252
It's not ok
1
value is 13790
No_duplication
It's ok
13790
0
value is 13790
It's ok
13790
[13790]
0
value is 13790
It's not ok
1
value is 23880
No_duplication
It's ok
23880
0
value is 21338
No_duplication
It's ok
21338
[23880, 21338]
0
value is 23880
It's not ok
1
value is 235700
No_duplication
It's ok
235700
0
value is 235700
It's ok
235700
[235700]
0
value is 235700
It's not ok
1
value is 575690
No_duplication
It's ok
575690
0
value is 26351
No_duplication
It's ok
26351
0
value is 6048
No_duplication
It's ok
6048
[575690, 26351, 6048]
0
value is 575690
It's not ok
1
value is 454456
No_duplication
It's ok
454456
0
value is 19845
No_duplication
It's ok
19845
0
value is 4761
No_duplication
It's ok
4761
[454456, 19845, 4761]
0
value is 454456
It's not ok

In [ ]:
#카테고리 국내, 해외, 항공호텔, 전체
PC_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/RFQ6OlJDT5-6MMKnjtJkcA/a10587421w80067081p84826225/")
PC_1.set_tab(2)
PC_1.set_value([1,1])
PC_1.get([1,2])

# 티몬 전체 카테고리
PC_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/ZcNefO8cTRe75IKFbPcCWw/a10587421w80067081p84826225/%3F_.useg%3D/")
PC_2.set_tab(1)
PC_2.set_value(1)
PC_2.get([1])

# # 여행 전체, 국내, 해외딜
# PC_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/hRoEL4w4RCyYpcmhDbvgWw/a10587421w80067081p84826225/%3F_.useg%3D/")
# PC_3.set_tab(3)
# PC_3.set_value([3,3,3])
# PC_3.get([1,2,3])
# PC_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "nope")
# PC_3.set_value([[3,3],[3,3],[3,3]])
# PC_3.get([1,2,3])

# 항공검색 국내, 해외
PC_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/KkWuVj5-SCOH7ddtrf4bOA/a10587421w80067081p84826225/")
PC_4.set_tab(2)
PC_4.set_value([2,2])
PC_4.get([1,2])

# 예약클릭 국내, 해외
PC_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/YspUfeZLQ7KyhFzRxehRcQ/a10587421w80067081p84826225/%3F_.useg%3D/")
PC_5.set_tab(2)
PC_5.set_value([2,2])
PC_5.get([1,2])

# B.com 카테고리/시스템/매뉴얼큐레이션/이벤트
PC_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/jQo-QVnGRrG1Y_Dtaoz0FQ/a10587421w80067081p84826225/%3F_.useg%3D/")
PC_6.set_tab(4)
PC_6.set_value([1,2,2,1])
PC_6.get([1,2,3,4])

#  홈화면 -> 딜상세
PC_7 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/S50YB2xDSLCbPXt17xrXBw/a10587421w80067081p84826225/%3F_u.date00%3D20170501%26_u.date01%3D20170507/")
PC_7.set_tab(2)
PC_7.set_value([2,[2,2]])
PC_7.get([1,2])

0
value is 3
It's not ok
1
value is 55821
No_duplication
It's ok
55821
0
value is 55821
It's ok
55821
[55821]
0
value is 55821
It's not ok
1
value is 28077
No_duplication
It's ok
28077
0
value is 28077
It's ok
28077
[28077]
0
value is 339903
No_duplication
It's ok
339903
0
value is 339903
It's ok
339903
[339903]
0
value is 10631
No_duplication
It's ok
10631
0
value is 6100
No_duplication
It's ok
6100
[10631, 6100]
0
value is 10631
It's not ok
1
value is 13459
No_duplication
It's ok
13459
0
value is 9202
No_duplication
It's ok
9202
[13459, 9202]
0
value is 13459
It's not ok
1
value is 10573
No_duplication
It's ok
10573
0
value is 1583
No_duplication
It's ok
1583
[10573, 1583]
0
value is 10573
It's not ok
1
value is 15548
No_duplication
It's ok
15548
0
value is 3506
No_duplication
It's ok
3506
[15548, 3506]
0
value is 15548
It's not ok
1
value is 1450
No_duplication
It's ok
1450
0
value is 1450
It's ok
1450
[1450]
0
value is 1450


In [10]:
# FIT_search_ad = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/Xb-8k7DlTNyQjJo3M5bMjw/a10587421w80067081p102782120/%3F_u.date00%3D20170501%26_u.date01%3D20170507/")
# FIT_search_ad.set_tab(2)
# FIT_search_ad.set_value([2,2])
# FIT_search_ad.set_date(7, 5)
# FIT_search_ad.get([1,2])
# FIT_search_iOS = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/p8shpiJuS4S2PNncs3XiDQ/a10587421w80067081p88823264/")
# FIT_search_iOS.set_tab(2)
# FIT_search_iOS.set_value([2,2])
# FIT_search_iOS.set_date(7, 5)
# FIT_search_iOS.get([1,2])
# FIT_search_Moweb = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/ERjO4gwrTtm8N0QyMBVMSw/a10587421w80067081p89357101/")
# FIT_search_Moweb.set_tab(2)
# FIT_search_Moweb.set_value([2,2])
# FIT_search_Moweb.set_date(7, 5)
# FIT_search_Moweb.get([1,2])

0
It's not ok
1
value is 1204236
No_duplication
It's ok
1204236
0
value is 7320
No_duplication
It's ok
7320
[1204236, 7320]
0
value is 7320
It's not ok
1
value is 1005
No_duplication
It's ok
1005
0
value is 1204236
It's ok
1204236
0
value is 1005
It's ok
1005
[1204236, 1005]
0
value is 1204236
It's not ok
1
value is 1149633
No_duplication
It's ok
1149633
0
value is 6969
No_duplication
It's ok
6969
[1149633, 6969]
0
value is 6969
It's not ok
1
value is 795
No_duplication
It's ok
795
0
value is 1149633
It's ok
1149633
0
value is 795
It's ok
795
[1149633, 795]
0
value is 1149633
It's not ok
1
value is 1077093
No_duplication
It's ok
1077093
0
value is 6087
No_duplication
It's ok
6087
[1077093, 6087]
0
value is 6087
It's not ok
1
value is 957
No_duplication
It's ok
957
0
value is 1077093
It's ok
1077093
0
value is 957
It's ok
957
[1077093, 957]
0
value is 1077093
It's not ok
1
value is 1080473
No_duplication
It's ok
1080473
0
value is 4388
No_duplication
It's ok
4388
[1080473, 4388]
0
value

In [12]:
PC_7.df

,161219 - 161225,161226 - 170101,170102 - 170108,170109 - 170115,170116 - 170122,170123 - 170129,170130 - 170205,170206 - 170212,170213 - 170219,170220 - 170226,170227 - 170305,170306 - 170312,170313 - 170319,170320 - 170326,170327 - 170402,170403 - 170409,170410 - 170416,170417 - 170423,170424 - 170430,170501 - 170507
PC_홈화면,1674353.0,1764618.0,1748564.0,1775214.0,1759362.0,1256796.0,1378383.0,1407449.0,1468144.0,1468572.0,1359602.0,1464013.0,1410815.0,1399869.0,1489399.0,1510200.0,1538806.0,1607849.0,1513224.0,1056712.0
PC_딜상세(국내),25879.0,15632.0,21225.0,15595.0,21666.0,8087.0,21813.0,20076.0,12754.0,18583.0,20579.0,22135.0,20868.0,22069.0,16205.0,15800.0,15591.0,6565.0,19971.0,6061.0
PC_딜상세(해외),2828.0,4438.0,3779.0,5695.0,4944.0,3901.0,4397.0,4664.0,7019.0,9901.0,2581.0,4844.0,3482.0,6463.0,7126.0,5322.0,3190.0,4634.0,3747.0,1567.0


In [8]:
pd.concat([FIT_search_ad.df, FIT_search_iOS.df, FIT_search_Moweb.df])

,170403 - 170409,170410 - 170416,170417 - 170423,170424 - 170430,170501 - 170507
로딩화면,17667.0,29924.0,30895.0,30618.0,20631.0
결과화면 랜딩,16573.0,28065.0,28862.0,27774.0,19031.0
로딩화면2,11418.0,36681.0,25312.0,22215.0,16855.0
결과화면 랜딩2,10922.0,34729.0,24700.0,21763.0,16665.0
로딩화면,8432.0,17768.0,14841.0,14896.0,10219.0
결과화면 랜딩,8080.0,16861.0,14040.0,13785.0,9554.0
로딩화면2,6597.0,29623.0,18109.0,16361.0,11274.0
결과화면 랜딩2,6355.0,28532.0,17820.0,16142.0,11101.0
로딩화면,3251.0,9917.0,10981.0,9545.0,5378.0
결과화면 랜딩,3031.0,9196.0,10121.0,8685.0,4916.0


In [15]:
pd.concat([And_6.df, And_7.df, iOS_6.df,iOS_7.df,Mo_6.df,PC_6.df])

,170417 - 170423,170424 - 170430,170501 - 170507
And 시스템 큐레이션 2. 주말특가 view,9308.0,9786.0,8685.0
And 시스템 큐레이션 2. 주말특가 클릭,1077.0,1037.0,867.0
And 카테고리 1. 여행홈 -> B.com(실시간 호텔),2678.0,2768.0,2419.0
And 카테고리 2. 여행홈 -> B.com(실시간 호텔),33.0,48.0,37.0
And 카테고리 3. 해외호텔 메인 검색모듈 view,4528.0,4567.0,4422.0
And 카테고리 3. 해외호텔 메인 검색모듈 -> 이동,3193.0,3204.0,2882.0
And 시스템 큐레이션 1. 여행홈 -> B.com,1345.0,1048.0,920.0
And_ 매뉴얼큐레이션 - 1.큐레이션 딜view,3635.0,3476.0,3265.0
매뉴얼큐레이션 - 1.큐레이션 딜 view -> 이동,912.0,932.0,835.0
And_이벤트 페이지,2993.0,3507.0,2002.0


In [12]:
#B.com 카테고리/시스템 큐레이션
And_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/urNzazUpRDu9q-17-qtmtw/a10587421w80067081p102782120/%3F_u.date00%3D20170501%26_u.date01%3D20170507/")
And_6.set_tab(5)
And_6.set_value([2,1,1,2,1])
And_6.set_date(7,3)
And_6.get([1,2,3,4,5])

#B.com 매뉴얼 큐레이션/이벤트
And_7 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/qWyiPek-Tsqgkok-rZsIjA/a10587421w80067081p102782120/%3F_u.date00%3D20170501%26_u.date01%3D20170507%26_r.tabId%3D210/")
And_7.set_tab(2)
And_7.set_value([2,2])
And_7.set_date(7,3)
And_7.get([1,2])

#B.com 카테고리/시스템 큐레이션
iOS_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/mzJgxll5SXqATSN_oHrUiQ/a10587421w80067081p88823264/")
iOS_6.set_tab(5)
iOS_6.set_value([2,1,1,2,1])
iOS_6.set_date(7,3)
iOS_6.get([1,2,3,4,5])

#B.com 매뉴얼 큐레이션/이벤트
iOS_7 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/tZRMJC4JRhm_X9Ql6C4KOg/a10587421w80067081p88823264/")
iOS_7.set_tab(2)
iOS_7.set_value([2,2])
iOS_7.set_date(7,3)
iOS_7.get([1,2])

#부킹닷컴 큐레이션, 호텔메인, 이벤트
Mo_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/W52iAReKSeydMYwmwkpBbA/a10587421w80067081p89357101/%3F_r.tabId%3D2101/")
Mo_6.set_tab(3)
Mo_6.set_value([2,2,2])
Mo_6.set_date(7,3)
Mo_6.get([1,2,3])

# B.com 카테고리/시스템/매뉴얼큐레이션/이벤트
PC_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/jQo-QVnGRrG1Y_Dtaoz0FQ/a10587421w80067081p84826225/%3F_u.date00%3D20170501%26_u.date01%3D20170507%26_r.tabId%3D4292/")
PC_6.set_tab(3)
PC_6.set_value([1,2,2,1])
PC_6.set_date(7,3)
PC_6.get([1,2,3,4])

0
It's not ok
1
value is 9308
No_duplication
It's ok
9308
0
value is 1077
No_duplication
It's ok
1077
[9308, 1077]
0
value is 9308
It's not ok
1
value is 2678
No_duplication
It's ok
2678
0
value is 2678
It's ok
2678
[2678]
0
value is 33
No_duplication
It's ok
33
0
value is 33
It's ok
33
[33]
0
value is 4528
No_duplication
It's ok
4528
0
value is 3193
No_duplication
It's ok
3193
[4528, 3193]
0
value is 1345
No_duplication
It's ok
1345
0
value is 1345
It's ok
1345
[1345]
0
value is 9786
No_duplication
It's ok
9786
0
value is 1037
No_duplication
It's ok
1037
[9786, 1037]
0
value is 2768
No_duplication
It's ok
2768
0
value is 2768
It's ok
2768
[2768]
0
value is 48
No_duplication
It's ok
48
0
value is 48
It's ok
48
[48]
0
value is 4567
No_duplication
It's ok
4567
0
value is 3204
No_duplication
It's ok
3204
[4567, 3204]
0
value is 1048
No_duplication
It's ok
1048
0
value is 1048
It's ok
1048
[1048]
0
value is 8685
No_duplication
It's ok
8685
0
value is 867
No_duplication
It's ok
867
[8685, 8

In [23]:
# #국내, 해외, 항공호텔, 전체카테고리
df1 = pd.concat([And_1.df, iOS_1.df, Mo_1.df, PC_1.df])

# #티몬 전체 카테고리
df2 = pd.concat([And_2.df, iOS_2.df, Mo_2.df, PC_2.df])

# #국내딜, 해외딜, 전체딜
df3 = pd.concat([And_3.df, iOS_3.df, Mo_3.df, PC_3.df])

# #항공검색 해외, 국내, 해외 -> 호텔메인, 해외 -> 국내메인
df4 = pd.concat([And_4.df, iOS_4.df, Mo_4.df, PC_4.df])

# #항공예약클릭 국내, 해외
df5 = pd.concat([And_5.df, iOS_5.df, Mo_5.df, PC_5.df])

# #부킹닷컴
df6 = pd.concat([And_6.df, And_7.df, iOS_6.df,iOS_7.df,Mo_6.df,PC_6.df])

# #홈 -> 딜상세(국내), 해외, 여행카테고리숏컷
df7 = pd.concat([And_8.df, iOS_8.df])

# #홈 -> 콜렉션(국내), 해외
df8 = pd.concat([And_9.df, iOS_9.df])

df9 = pd.concat([And_10.df, iOS_10.df])

In [21]:
# #######월간 여행딜

# #국내, 해외, 여행전체 딜
# And_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/c9agY1srRx2PaD8JAkK0yw/a10587421w80067081p102782120/")
# And_3.set_tab(3)
# And_3.set_value([3,3,3])
# And_3.get([1,2,3])

# And_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "off")
# And_3.set_value([[3,3],[3,3],[3,3]])
# And_3.get([1,2,3])

# #국내딜, 해외딜, 전체딜
# iOS_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/A9m3wyZgQTmxJrO3lQ6UMw/a10587421w80067081p88823264/%3F_u.date00%3D20170327%26_u.date01%3D20170402%26_r.tabId%3D4646%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ/")
# iOS_3.set_tab(3)
# iOS_3.set_value([3,3,3])
# iOS_3.get([1,2,3])

# iOS_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "off")
# iOS_3.set_value([[3,3],[3,3],[3,3]])
# iOS_3.get([1,2,3])

# #여행딜 전체, 국내, 해외
# Mo_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/upJUz96ERVGUIE2WWd8Tgg/a10587421w80067081p89357101/")
# Mo_3.set_tab(3)
# Mo_3.set_value([3,3,3])
# Mo_3.get([1,2,3])

# Mo_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "off")
# Mo_3.set_value([[3,3],[3,3],[3,3]])
# Mo_3.get([1,2,3])

# # 여행 전체, 국내, 해외딜
# PC_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/hRoEL4w4RCyYpcmhDbvgWw/a10587421w80067081p84826225/%3F_.useg%3D/")
# PC_3.set_tab(3)
# PC_3.set_value([3,3,3])
# PC_3.get([1,2,3])

# PC_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "off")
# PC_3.set_value([[3,3],[3,3],[3,3]])
# PC_3.get([1,2,3])

In [ ]:
#부킹닷컴 큐레이션, 호텔배너, 주말특가, 검색모듈
And_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/urNzazUpRDu9q-17-qtmtw/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D/")
And_6.set_tab(4)
And_6.set_value([2,1,2,2])
And_6.get([1,2,3,4])
#부킹닷컴 큐레이션, 호텔배너, 주말특가, 해외호텔검색모듈
iOS_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/fiA5QbsjQAeJxkmaYUFJEQ/a10587421w80067081p88823264/")
iOS_6.set_tab(4)
iOS_6.set_value([2,1,2,2])
iOS_6.get([1,2,3,4])
#부킹닷컴 큐레이션, 호텔메인
Mo_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/W52iAReKSeydMYwmwkpBbA/a10587421w80067081p89357101/")
Mo_6.set_tab(2)
Mo_6.set_value([2,2])
Mo_6.get([1,2])
# 부킹닷컴 큐레이션, 해외호텔 카테고리, 주말특가
PC_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/jQo-QVnGRrG1Y_Dtaoz0FQ/a10587421w80067081p84826225/%3F_.useg%3D/")
PC_6.set_tab(3)
PC_6.set_value([2,1,2])
PC_6.get([1,2,3])

In [14]:
# booking_df_User = pd.concat([And_6.df, iOS_6.df, Mo_6.df, PC_6.df])

In [49]:
# #######월간항공

# #항공검색 해외, 국내, 해외->호텔메인, 해외->국내항공메인
# And_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/5fyYOU-QTN6tQSwRlObkUw/a10587421w80067081p102782120/%3F_u.date00%3D20170101%26_u.date01%3D20170301%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ/")
# And_4.set_tab(4)
# And_4.set_value([2,2,[2,2],[2,2]])
# And_4.get([1,2,3,4])

# #항공예약 클릭 국내, 해외 
# And_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/_yDrAZzXTaeMOGlYkwqqbw/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D%26_r.tabId%3D6028/")
# And_5.set_tab(2)
# And_5.set_value([1,1])
# And_5.get([1,2])

# #항공검색 해외, 국내, 해외 -> 호텔메인, 해외 -> 국내메인
# iOS_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/2K60xdJeT2isKsPe4tgpEQ/a10587421w80067081p88823264/%3F_u.date00%3D20170301%26_u.date01%3D20170320/")
# iOS_4.set_tab(4)
# iOS_4.set_value([2,2,[2,2],[2,2]])
# iOS_4.get([1,2,3,4])

# #항공예약클릭 국내, 해외
# iOS_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/btj6gUJmQ2-NZsSB1vB3tw/a10587421w80067081p88823264/%3F_u.date00%3D20170301%26_u.date01%3D20170331%26_.useg%3D/")
# iOS_5.set_tab(2)
# iOS_5.set_value([1,1])
# iOS_5.get([1,2])

# #항공검색 해외, 국내, 해외->호텔, 해외->국내
# Mo_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/7aMTsZECQu2GVnxwp3IXRg/a10587421w80067081p89357101/")
# Mo_4.set_tab(4)
# Mo_4.set_value([2,2,[2,2],[2,2]])
# Mo_4.get([1,2,3,4])

# #항공예약클릭 국내, 해외
# Mo_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/Shs74NpmR1Wcy5isHt_ZGA/a10587421w80067081p89357101/")
# Mo_5.set_tab(2)
# Mo_5.set_value([1,1])
# Mo_5.get([1,2])

# # 항공검색 국내, 해외
# PC_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/KkWuVj5-SCOH7ddtrf4bOA/a10587421w80067081p84826225/")
# PC_4.set_tab(2)
# PC_4.set_value([2,2])
# PC_4.get([1,2])

# # 예약클릭 국내, 해외
# PC_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/YspUfeZLQ7KyhFzRxehRcQ/a10587421w80067081p84826225/%3F_.useg%3D/")
# PC_5.set_tab(2)
# PC_5.set_value([2,2])
# PC_5.get([1,2])

In [48]:
# pd.concat([And_4.df, And_5.df, iOS_4.df, iOS_5.df, Mo_4.df, Mo_5.df, PC_4.df, PC_5.df])